In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime

file_path = 'output.xlsx'
maxpage = 2                                                         # сколько страниц топа грузим
page_size = 60                                                      # число моделей на странице

# Чтение данных
data = pd.read_excel(file_path)

# Дополнительные столбцы
data['Ranking'] = maxpage*page_size - data['N'] + 1
data['Unique_Title'] = data['Link'].apply(lambda x: x.split('/')[-1])
data['Max_Version'] = data['Platform'].apply(lambda x: x.split(' + ')[0] if ' + ' in x else None)
data['File_Type'] = data['Platform'].apply(lambda x: x.split(' + ')[1] if ' + ' in x else None)

data['Pubdate'] = pd.to_datetime(data['Pubdate'])
data['Full_Category'] = data['Category'] + '.' + data['Subcategory']

current_date = datetime.now()
data['Duration'] = (current_date - data['Pubdate']).dt.days

data.drop(['N','Link', 'Standard', 'Pubdate', 'Platform'], axis=1, inplace=True)


In [2]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Curdate        7920 non-null   object 
 1   Title          7920 non-null   object 
 2   Category       7920 non-null   object 
 3   Subcategory    7920 non-null   object 
 4   Corona         7854 non-null   object 
 5   V-Ray          5082 non-null   object 
 6   Username       7920 non-null   object 
 7   Followers      7920 non-null   int64  
 8   Selected       7913 non-null   float64
 9   Ranking        7920 non-null   int64  
 10  Unique_Title   7920 non-null   object 
 11  Max_Version    7881 non-null   object 
 12  File_Type      7881 non-null   object 
 13  Full_Category  7920 non-null   object 
 14  Duration       7920 non-null   int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 928.2+ KB


None

In [3]:
fig1 = px.line(data, x='Curdate', y='Ranking', color='Unique_Title', title='Top 120 Rank Dynamics')
fig1.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [4]:
data_cat_gr = data.groupby(by='Category', as_index=False)['Ranking'].sum()
data_cat_gr.columns = ['Category', 'Total_Ranking']
data_cat_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)

fig2 = px.bar(data_cat_gr, x='Category', y='Total_Ranking', title='Top 120 total Rank by Category')
fig2.show()


In [5]:
data_subcat_gr = data.groupby(by=['Full_Category', 'Category'], as_index=False)['Ranking'].sum()
data_subcat_gr.columns = ['Full_Category', 'Category', 'Total_Ranking']
data_subcat_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)

cat_list = data_subcat_gr['Full_Category'].tolist()
cat_dict = {'Full_Category': cat_list}

fig3 = px.bar(data_subcat_gr, x='Full_Category', y='Total_Ranking', 
              category_orders=cat_dict, color='Category', height=600, title='Top 120 total Rank by Subcategory')
fig3.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [6]:
data_unique_gr = data.groupby(by=['Unique_Title', 'Category', 'Username'], as_index=False)['Ranking'].sum()
data_unique_gr.columns = ['Unique_Title', 'Category', 'Username', 'Total_Ranking']
data_unique_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)
data_unique_gr = data_unique_gr.head(30)

cat_list = data_unique_gr['Unique_Title'].tolist()
cat_dict = {'Unique_Title': cat_list}

fig4 = px.bar(data_unique_gr, x='Unique_Title', y='Total_Ranking', hover_data='Username',
              category_orders=cat_dict, color='Category', height=800, title='Top 120 total Rank by Unique_Title')
fig4.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [7]:
data_user_gr = data.groupby(by='Username', as_index=False)['Ranking'].sum()
data_user_gr.columns = ['Username', 'Total_Ranking']
data_user_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)
data_user_gr = data_user_gr.head(30)

cat_user_list = data_user_gr['Username'].tolist()
cat_user_dict = {'Username': cat_user_list}

fig5 = px.bar(data_user_gr, x='Username', y='Total_Ranking', 
              category_orders=cat_user_dict, height=800, title='Top 120 total Rank by Username')
fig5.show()

In [8]:
data_max_file = data[['Unique_Title', 'Max_Version', 'File_Type']].drop_duplicates().reset_index()
data_max_file_pivot = data_max_file.pivot_table(
    values='Unique_Title',
    index='Max_Version',
    columns='File_Type',
    aggfunc='count'
)

fig6 = px.imshow(data_max_file_pivot, color_continuous_scale='Blues', 
                 title='Top 120 distribution by Max_Version and File_Type')
fig6.show()

In [10]:
fig1.write_html("Top 120 Rank Dynamics.html")
fig2.write_html("Top 120 total Rank by Category.html")
fig3.write_html("Top 120 total Rank by Subcategory.html")
fig4.write_html("Top 120 total Rank by Unique_Title.html")
fig5.write_html("Top 120 total Rank by Username.html")
fig6.write_html("Top 120 distribution by Max_Version and File_Type.html")

In [9]:
display(data[data['Username']=='Juliet-t-e'])

,Curdate,Title,Category,Subcategory,Corona,V-Ray,Username,Followers,Selected,Ranking,Unique_Title,Max_Version,File_Type,Full_Category,Duration
1190,2024-12-05,Entrance to a guarded area/parking lot,Architecture,Urban environment,Corona,NaN,Juliet-t-e,9,77.0,10,vezd_na_okhraniaemuiu_territoriiu_parkovku,3dsMax 2017,obj,Architecture.Urban environment,283
1309,2024-12-06,Entrance to a guarded area/parking lot,Architecture,Urban environment,Corona,NaN,Juliet-t-e,9,77.0,11,vezd_na_okhraniaemuiu_territoriiu_parkovku,3dsMax 2017,obj,Architecture.Urban environment,283
1429,2024-12-07,Entrance to a guarded area/parking lot,Architecture,Urban environment,Corona,NaN,Juliet-t-e,9,79.0,11,vezd_na_okhraniaemuiu_territoriiu_parkovku,3dsMax 2017,obj,Architecture.Urban environment,283
1550,2024-12-08,Entrance to a guarded area/parking lot,Architecture,Urban environment,Corona,NaN,Juliet-t-e,9,79.0,10,vezd_na_okhraniaemuiu_territoriiu_parkovku,3dsMax 2017,obj,Architecture.Urban environment,283
1645,2024-12-09,Entrance to a guarded area/parking lot,Architecture,Urban environment,Corona,NaN,Juliet-t-e,9,79.0,35,vezd_na_okhraniaemuiu_territoriiu_parkovku,3dsMax 2017,obj,Architecture.Urban environment,283
1784,2024-12-11,Entrance to a guarded area/parking lot,Architecture,Urban environment,Corona,NaN,Juliet-t-e,9,79.0,16,vezd_na_okhraniaemuiu_territoriiu_parkovku,3dsMax 2017,obj,Architecture.Urban environment,283
1883,2024-12-12,Entrance to a guarded area/parking lot,Architecture,Urban environment,Corona,NaN,Juliet-t-e,9,80.0,37,vezd_na_okhraniaemuiu_territoriiu_parkovku,3dsMax 2017,obj,Architecture.Urban environment,283
